## import libraries

In [1]:
from compas.geometry import Frame, Plane
from compas.geometry import Box, Sphere
from compas_vol.primitives import VolBox, VolSphere
from compas_vol.combinations import Union, SmoothUnion, Intersection, SmoothUnionList
from compas_vol.modifications import Shell

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import get_random_vector_3D, bbox_edges

## create volumetric object (CSG tree)

In [3]:
#generate random points around a sphere
nbrPts = 20
sphereRad = 3.4
pts = [get_random_vector_3D(sphereRad) for _ in range(nbrPts)]
print(pts)

[(-2.7610381680420586, -1.6375064690647476, 1.120375293545005), (0.7313615624028013, -2.9617948264811527, -1.5009602495966408), (-1.0420622785111724, -1.3117391278519677, 2.9586224274425867), (0.2906069288595286, -1.2560482325846047, -3.1460912972003725), (-1.1949934611638233, 2.7643008018819146, -1.5781735343398415), (2.2733307377325245, -0.8170545039136314, -2.3925696007671218), (-0.9862064738478581, -2.4452807395553124, -2.146625001182181), (-0.5400449392396329, -1.3423242105474604, 3.07677057600657), (-2.0216625240762007, 2.3586033648719207, -1.3819807545551055), (-1.4718110694901683, -0.1698400281247205, -3.060216747319183), (-1.370798988291391, -1.3005376572585456, 2.826572471342589), (-2.491302108292523, -1.9652373085842796, 1.2211699825027003), (-0.9056644576145831, -3.2300916192915228, 0.5534257142529154), (-0.15778942862743822, 2.7078937776521825, -2.049978971883863), (-0.4717607178887718, 3.0281131049381793, 1.4724037655340085), (-1.3060147912405118, -0.0402483008196528, 3.1

In [4]:
#create volumetric boxes and unify them
volBoxes = []
for [x,y,z] in pts:
    plane = Plane([x,y,z],[x,y,z])
    box = Box(Frame.from_plane(plane),1.2,1.2,3.5)
    vbox = VolBox(box, 0.1)
    volBoxes.append(vbox)
uvb = Union(volBoxes)

In [25]:
#combine core and branches
vs = VolSphere(Sphere([0,0,0], 2.0))
# su = SmoothUnion(uvb, vs, 2.0)
su = SmoothUnionList([uvb, vs], 10.0)

In [26]:
#thicken shell and cull extremities
s = Shell(su, 0.2, 1.0)
node = Intersection(VolSphere(Sphere([0,0,0], 4.8)), s)

## workspace (dense grid)

In [27]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -5.0, 5.0
lby, uby = -5.0, 5.0
lbz, ubz = -5.0, 5.0
# resolution(s)
nx, ny, nz = 120, 120, 120
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [28]:
dm = node.get_distance_numpy(x, y, z)

## generate isosurface (marching cube)

In [29]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [30]:
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = bbox_edges(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))
p.add_points(np.array([[lbx,lby,lbz]]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.047710…

2